In [ ]:
import numpy as np
import pathlib
import os
import math
from skimage import io
from skimage.metrics import structural_similarity as ssim
import cv2  

In [ ]:
def calculate_ssim_and_psnr(image1_path, image2_path):
    # Read the two images
    image1 = io.imread(image1_path)
    image2 = io.imread(image2_path)

    # Convert images to grayscale (if they are not already)
    if len(image1.shape) == 3:
        image1 = image1[:,:,0]
    if len(image2.shape) == 3:
        image2 = image2[:,:,0]

    # Calculate SSIM
    ssim_value = ssim(image1, image2, data_range=2**8-1) 
    
    # Calculate PSNR
    def calculate_psnr(img1, img2):
        # img1 and img2 have range [0, 255], 8 bit
        img1 = img1.astype(np.float64)
        img2 = img2.astype(np.float64)
        mse = np.mean((img1 - img2)**2)
        if mse == 0:
            return float('inf')
        return 20 * math.log10(255 / math.sqrt(mse))        
    
    psnr_value = calculate_psnr(image1, image2)

    return ssim_value, psnr_value

In [ ]:
# Set the folder path where images are located and the tile size
path = pathlib.Path(os.getcwd())
FMD_testmix = path/'FMD_testmix'
noise_levels = ['raw','avg2','avg4','avg8','avg16']

In [ ]:
# get metrics for original noise images
all_ssim = []
all_psnr = []
for i,levels in enumerate(noise_levels):
    files = sorted(os.listdir(FMD_testmix/levels))
    for j,fn in enumerate(files):
       temp1, temp2 = calculate_ssim_and_psnr(FMD_testmix/levels/fn,FMD_testmix/'gt'/fn)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(files)-1:
           print(f'{levels} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

In [ ]:
# get metrics for model predictions
all_ssim = []
all_psnr = []
for i,levels in enumerate(noise_levels):
    files = sorted(os.listdir(FMD_testmix/'denoised_images'/levels))
    for j,fn in enumerate(files):
       temp1, temp2 = calculate_ssim_and_psnr(FMD_testmix/'denoised_images'/levels/fn,FMD_testmix/'gt'/fn)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(files)-1:
           print(f'{levels} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []